### Imports

In [1]:
# import reddit api wrapper
import praw
import pandas as pd
import datetime
import sqlite3

from textblob import TextBlob
import altair as alt

# Path hack
import os
# change directory from the current Analysis folder to the top level folder for easier navigation
os.chdir('../')
# confirm we're at /RedditTextAnalysis
print(os.getcwd())

# note that the %load_ext autoreload line only needs to be be run once
%load_ext autoreload

/Users/prcork/MiscDataProjects/collaboration/RedditTextAnalysis


In [2]:
# by including this autoreload command, we only need to re-import Spatial_Joins if we make/save changes to the original py file
%autoreload
from Functions.nfl_gamethreads import nfl_gamethreads

### Set Up Reddit Instance

In [3]:
# access password and client secret id via local files
with open('pw.txt', 'r') as file1:
    pw = file1.read()

with open('client_secret.txt', 'r') as file2:
    cs = file2.read()

# create a praw Reddit instance with app credentials and secret info passed through
reddit = praw.Reddit(
    client_id="XbesrQBvKymjgLdgg_D6lA",
    client_secret=cs,
    user_agent="NFLTextAnalysis/0.0.1",
    username="ta_api",
    password=pw
)

# store an instance of the Redditor object for the gamethread account
nfl_bot = reddit.redditor("nfl_gamethread")

### Get Raw Data

In [4]:
gamethread_list = nfl_gamethreads.get_submissions(nfl_bot, 1000)

### Transform in Gamethread DataFrame

In [20]:
gamethread_df = nfl_gamethreads.build_gamethread_df(reddit, gamethread_list, season=2021)

### Store Data in CSV & SQL

In [13]:
gamethread_df.rename(columns={"id":"submission_id", "home_team_full":"home_team",
"away_team_full":"away_team", "diff":"score_difference", "pred_winner":"predicted_winner",
"pred_ou":"predicted_over_under", "pred_diff":"predicted_difference"}, inplace=True)

gamethread_df['season'] = 2021

In [14]:
gamethread_df

,submission_id,title,date,week,home_team,home_team_wins,home_team_losses,home_team_ties,away_team,away_team_wins,...,away_team_ties,home_score,away_score,winner,combined_score,score_difference,predicted_winner,predicted_difference,predicted_over_under,season
0,pwto53,Game Thread: Philadelphia Eagles (1-1) at Dall...,2021-09-27,3,Dallas Cowboys,1,1,0,Philadelphia Eagles,1,...,0,41,21,Cowboys,62,20,Dallas,3.5,51.5,2021
1,q1ih03,Game Thread: Las Vegas Raiders (3-0) at Los An...,2021-10-04,4,Los Angeles Chargers,2,1,0,Las Vegas Raiders,3,...,0,28,14,Chargers,42,14,Los Angeles,3.0,51.5,2021
2,q0tsv6,Game Thread: Tampa Bay Buccaneers (2-1) at New...,2021-10-03,4,New England Patriots,1,2,0,Tampa Bay Buccaneers,2,...,0,17,19,Buccaneers,36,2,New England,7.0,49.5,2021
3,q0pdqs,Game Thread: Baltimore Ravens (2-1) at Denver ...,2021-10-03,4,Denver Broncos,3,0,0,Baltimore Ravens,2,...,0,7,23,Ravens,30,16,Denver,1.0,44.0,2021
4,q0pdpz,Game Thread: Pittsburgh Steelers (1-2) at Gree...,2021-10-03,4,Green Bay Packers,2,1,0,Pittsburgh Steelers,1,...,0,27,17,Packers,44,10,Green Bay,6.5,45.5,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,rzwo9f,Game Thread: Washington Football Team (6-10) a...,2022-01-09,18,New York Giants,4,12,0,Washington Football Team,6,...,0,7,22,Football Team,29,15,New York,6.0,36.5,2021
209,rzwo81,Game Thread: Indianapolis Colts (9-7) at Jacks...,2022-01-09,18,Jacksonville Jaguars,2,14,0,Indianapolis Colts,9,...,0,26,11,Jaguars,37,15,Jacksonville,14.0,43.5,2021
210,rzwo5r,Game Thread: Green Bay Packers (13-3) at Detro...,2022-01-09,18,Detroit Lions,2,13,1,Green Bay Packers,13,...,0,37,30,Lions,67,7,Detroit,4.0,45.0,2021
211,rzez6y,Game Thread: Dallas Cowboys (11-5) at Philadel...,2022-01-09,18,Philadelphia Eagles,9,7,0,Dallas Cowboys,11,...,0,26,51,Cowboys,77,25,Philadelphia,6.0,46.5,2021


In [15]:
# TODO: Update for ingesting 2022 season to append to a csv rather than write a new one
gamethread_df.to_csv("Data/gamethreads_2021.csv", index=False)

In [17]:
with sqlite3.connect("nfl_gamethreads.db") as conn:
    gamethread_df.to_sql(name='gamethreads', con=conn, if_exists='append', index=False)